#### At first let us ingest the datasets

#### Click the red '1010' icon in the right top of the toolbar. There customer_usage_history.csv will appear. Click on th arrow at the right of Insert to Code. And then select Insert Spark DataFrame in Python. This will insert the dataframe in the next cell. Change the name of the dataframe (in both theplaces) to customer_usage_history and run the cell.

In [1]:


import os
from pyspark.sql import SQLContext
# Add asset from file system
customer_usage_history = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/customer_usage_history.csv', header='true', inferSchema = 'true')
customer_usage_history.show(5)





+---+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+-----+
| ID|LONGDISTANCE|INTERNATIONAL|LOCAL|DROPPED|PAYMETHOD|LOCALBILLTYPE|LONGDISTANCEBILLTYPE|USAGE|RATEPLAN|CHURN|
+---+------------+-------------+-----+-------+---------+-------------+--------------------+-----+--------+-----+
|  1|          23|            0|  206|      0|       CC|       Budget|      Intnl_discount|  229|       3|    T|
|  6|          29|            0|   45|      0|       CH|    FreeLocal|            Standard|   75|       2|    F|
|  8|          24|            0|   22|      0|       CC|    FreeLocal|            Standard|   47|       3|    F|
| 11|          26|            0|   32|      1|       CC|       Budget|            Standard|   59|       1|    F|
| 17|          12|            0|   46|      4|       CC|    FreeLocal|            Standard|   58|       1|    F|
+---+------------+-------------+-----+-------+---------+-------------+--------------------+-----

#### Repeat the able step for the seconf dataset - customer_demograhic_history.csv. This time change the name of the dataframe to customer_demograhic_history

In [4]:


# Add asset from file system
customer_demographic_history = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/customer_demographic_history.csv', header='true', inferSchema = 'true')
customer_demographic_history.show(5)





+---+------+------+--------+---------+--------+---------+
| ID|GENDER|STATUS|CHILDREN|ESTINCOME|CAROWNER|      AGE|
+---+------+------+--------+---------+--------+---------+
|  1|     F|     S|       1|  38000.0|       N|24.393333|
|  6|     M|     M|       2|  29616.0|       N|49.426667|
|  8|     M|     M|       0|  19732.8|       N|50.673333|
| 11|     M|     S|       2|    96.33|       N|56.473333|
| 17|     M|     M|       2|  53010.8|       N|    18.84|
+---+------+------+--------+---------+--------+---------+
only showing top 5 rows



#### Now let us create 2 temporary views from these two dataframes so that we can join these two table to get the merged dataset. We shall join these two datasets using the column 'ID'

In [13]:
customer_usage_history.createOrReplaceTempView("custusagehistT")

In [14]:
customer_demographic_history.createOrReplaceTempView("custdemohistT")

#### Now we shall join these two views using the column ID. That will create the dataframe that we can use for building the model

In [16]:
enhacced_customer_history = \
SQLContext(sc).sql("select a.*, b.GENDER, b.STATUS, b.CHILDREN, b.ESTINCOME, b.CAROWNER, b.AGE from custusagehistT a,custdemohistT b where a.ID = b.ID")

#### Let us create another temporary view out if this merged dataframe and see how many records we have with Churn Flag 'Y' and how many with Churn Flaf 'N'

In [17]:
enhacced_customer_history.createOrReplaceTempView("enhancedcustT")

In [18]:
SQLContext(sc).sql("select CHURN, count(CHURN) as churnClount from enhancedcustT group by CHURN").show()

+-----+-----------+
|CHURN|churnClount|
+-----+-----------+
|    F|        838|
|    T|        577|
+-----+-----------+



#### Now let us save this merged datafrme as a CSV file. This file we shall read back in next notebook to build tyhe model. Please note that the file saved this way would be saved in spark format where the main data would be saved as a part file. However while reading the data back in Spark you can use the same name that you have used to save the file.

In [19]:
enhacced_customer_history.coalesce(1).write.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/enhanced_customers.csv', header='true')